<a href="https://colab.research.google.com/github/dwhew/data_science/blob/main/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What is the nicest city to live in?

Notes
*   warmest?
*   least rain?
*   has snow?
*   score based on all attributes would be the most objective (maybe number of days could be a good counter and then rank/sort or use index. must prioritise attributes)






In [1]:
!pip install -U plotly

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.14.3)


In [2]:
import pandas as pd
import plotly.express as px

pd.options.plotting.backend = 'plotly'


## prelim exploration of data

In [3]:
inndf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Innsbruck_weather_2015-19.csv")
inndf.head()

,date,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),month,month-year,year
0,2015-01-01,-3.137500,-7.4,1.0,724.241667,92.458333,1.166667,0.0,Jan,Jan-2015,2015
1,2015-01-02,-3.341667,-9.2,1.5,721.975000,94.875000,1.125000,1.6,Jan,Jan-2015,2015
2,2015-01-03,1.458333,0.4,3.2,717.537500,96.833333,1.791667,13.1,Jan,Jan-2015,2015
3,2015-01-04,2.108333,-0.2,4.6,717.608333,79.291667,3.041667,25.9,Jan,Jan-2015,2015
4,2015-01-05,-0.841667,-5.3,2.4,721.662500,74.875000,2.666667,0.3,Jan,Jan-2015,2015


In [4]:
len(inndf)/365

5.002739726027397

In [5]:
inndf['year'].unique()

array([2015, 2016, 2017, 2018, 2019])

In [6]:
len(inndf.loc[inndf['min_temp (deg C)'] < 0])/5

99.8

In [7]:
inndf.dtypes

date                   object
mean_temp (deg C)     float64
min_temp (deg C)      float64
max_temp (deg C)      float64
mean_pressure (mb)    float64
mean_humidity (%)     float64
mean_wind (ms-1)      float64
precip_sum (mm)       float64
month                  object
month-year             object
year                    int64
dtype: object

In [8]:
inndf['year'].value_counts()

2016    366
2019    365
2018    365
2017    365
2015    365
Name: year, dtype: int64

 # load  city data

In [9]:
#inndf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Innsbruck_weather_2015-19.csv")
londf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/London_weather_2015-19.csv")
sindf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Singapore_weather_2015-19.csv')
syddf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Sydney_weather_2015-19.csv')
tehdf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Tehran_weather_2015-19.csv')

#assign dfs to list for later iterating
dflist = [inndf, londf, sindf, syddf, tehdf]

#assign cities to a list for later use
citylist = ['Innsbruck', 'London', 'Singapore', 'Sydney', 'Tehran']

In [10]:
for df, name in zip(dflist, citylist):
  df['city'] = name
lord_of_lists = pd.concat(dflist)
lord_of_lists

,date,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),month,month-year,year,city
0,2015-01-01,-3.137500,-7.4,1.0,724.241667,92.458333,1.166667,0.0,Jan,Jan-2015,2015,Innsbruck
1,2015-01-02,-3.341667,-9.2,1.5,721.975000,94.875000,1.125000,1.6,Jan,Jan-2015,2015,Innsbruck
2,2015-01-03,1.458333,0.4,3.2,717.537500,96.833333,1.791667,13.1,Jan,Jan-2015,2015,Innsbruck
3,2015-01-04,2.108333,-0.2,4.6,717.608333,79.291667,3.041667,25.9,Jan,Jan-2015,2015,Innsbruck
4,2015-01-05,-0.841667,-5.3,2.4,721.662500,74.875000,2.666667,0.3,Jan,Jan-2015,2015,Innsbruck
...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2019-12-27,9.550000,4.4,14.2,659.087500,43.250000,1.375000,0.0,Dec,Dec-2019,2019,Tehran
1822,2019-12-28,8.437500,5.6,12.0,660.025000,56.750000,5.125000,2.0,Dec,Dec-2019,2019,Tehran
1823,2019-12-29,7.900000,3.5,12.6,663.150000,40.000000,3.125000,0.0,Dec,Dec-2019,2019,Tehran
1824,2019-12-30,8.000000,3.0,12.4,664.400000,34.500000,2.750000,0.0,Dec,Dec-2019,2019,Tehran


In [11]:
lord_of_lists.describe()

,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),year
count,9125.000000,9125.000000,9125.000000,9125.000000,9125.000000,9124.000000,9125.000000,9125.000000
mean,17.595085,13.501545,22.431858,730.240644,66.063253,3.583339,4.305425,2017.000548
std,9.136516,9.306446,9.369491,39.624290,20.780733,1.895595,12.387127,1.413710
min,-11.487500,-17.000000,-7.800000,650.250000,7.500000,0.000000,0.000000,2015.000000
25%,10.822727,6.400000,15.400000,708.354167,57.083333,2.125000,0.000000,2016.000000
50%,17.779167,13.200000,23.200000,755.462500,72.375000,3.125000,0.000000,2017.000000
75%,26.600000,22.500000,31.000000,759.812500,80.750000,4.620536,2.300000,2018.000000
max,37.600000,33.800000,43.700000,780.487500,100.000000,18.166667,188.300000,2019.000000


#histograms

In [12]:
fig = px.histogram()

for df, city in zip(dflist, citylist):
  fig.add_histogram(x=df['mean_temp (deg C)'], 
                    name = city, 
                    xbins={'size':1}
                    )

fig.update_layout(
    barmode='overlay'
    title="Histogram of mean temperatures (deg C)",
    xaxis_title="Temperature (deg C)",
    yaxis_title="Number of days",
    legend_title="City",
    )

fig.update_traces(opacity=0.75)

fig.show()

In [13]:
fig = px.box(lord_of_lists, y='mean_temp (deg C)', x='city')
fig.show()




#fig.update_layout(barmode='overlay')
#fig.update_layout(
    #title="Histogram of mean temperatures",
    #xaxis_title="Temperature (deg C)",
    #yaxis_title="Number of days",
    #legend_title="Legend",
    #)


# calculate number of days per categorie e.g. days of no rain.
Idea for what to use
-no rain 
-mean temp > 15
-min temp > 0

In [14]:
len(inndf[inndf['precip_sum (mm)'] == 0.0]) /5


165.8

In [43]:
# Here I iterated through a list of the dataframes 'dflist'.  

NoRain = []     #no. of days without rain (annual average)
AboveZero = []  #no. of days with min temp above 0
NotWindy = []   #no. of days without srong wind

for df in dflist:
  NoRain.append(len(df[df['precip_sum (mm)'] == 0.0]) /5)
  AboveZero.append(len(df[df['min_temp (deg C)'] > 0]) /5)
  NotWindy.append(len(df[df['mean_wind (ms-1)'] < 4]) /5)

#this would then be added to a dataframe or just plotted


In [41]:
#or, create a dict
stats_list = [NoRain, AboveZero, NotWindy]
stats_listSTR = ['NoRain', 'AboveZero', 'NotWindy']
citystats = {}

for name, stat in zip(stats_listSTR, stats_list):

    citystats[name] = stat


citystats
  

{'AboveZero': [263.8, 342.2, 365.2, 365.2, 350.6],
 'NoRain': [165.8, 198.0, 183.6, 205.6, 308.8],
 'NotWindy': [0.0, 2.0, 0.0, 8.8, 0.2]}

In [42]:
#convert to DataFrame and plot    https://plotly.com/python/pandas-backend/#:~:text=Since%20version%200.25%2C%20Pandas%20has,even%20needing%20to%20import%20Plotly.

cdf = pd.DataFrame.from_dict(data= citystats, orient='columns')
cdf['city'] = citylist

cdf = cdf.set_index('city')
fig = cdf.plot.bar(facet_row='variable', title='Total days of...', labels='days')
fig.show()

note for later: barometric pressure over 1022.689 mb indicates clear and stable (google.com)
and windspeed less than 4 

# Aproach 2. group by months

In [31]:
#get average day by month 
#will set cumulative for rainfall in next step

month_ordered = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

inndf_months = inndf.groupby(by=['month']).mean().reindex(month_ordered)
londf_months = londf.groupby(by=['month']).mean().reindex(month_ordered)
sindf_months = sindf.groupby(by=['month']).mean().reindex(month_ordered)
syddf_months = syddf.groupby(by=['month']).mean().reindex(month_ordered)
tehdf_months = tehdf.groupby(by=['month']).mean().reindex(month_ordered)

mdflist = [inndf_months, londf_months, sindf_months, syddf_months, tehdf_months]

#and replace precip_sum (mm) with groupby().sum()



In [32]:
inndf_months

,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),year
month,,,,,,,,
Jan,-1.036986,-4.572258,3.523226,710.995404,84.563344,2.028521,7.080000,2017.000000
Feb,1.459536,-2.696454,7.192199,710.668417,77.120303,2.071454,3.158865,2016.992908
Mar,6.061640,0.803871,12.434839,709.293662,69.261675,2.447647,4.507097,2017.000000
Apr,10.601215,4.645333,16.982667,710.002559,64.274459,2.805385,5.587333,2017.000000
May,14.001605,8.596129,20.202581,709.830970,69.726098,2.369585,10.334839,2017.000000
Jun,18.983879,13.187333,25.584000,711.072269,68.044504,2.257706,11.106667,2017.000000
Jul,19.910417,14.322581,26.470323,711.496566,70.569659,1.934507,11.485806,2017.000000
Aug,19.554905,13.970323,26.247097,712.556892,72.964836,2.022288,14.955484,2017.000000
Sep,14.570158,9.645333,20.586000,712.863897,76.828923,1.865618,9.295333,2017.000000


In [28]:
inndf_months['precip_sum (mm)'] = inndf[['precip_sum (mm)', 'month']].groupby(by=['month']).sum().reindex(month_ordered)['precip_sum (mm)']

In [29]:
inndf_months

,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),year
month,,,,,,,,
Jan,-1.036986,-4.572258,3.523226,710.995404,84.563344,2.028521,1097.4,2017.000000
Feb,1.459536,-2.696454,7.192199,710.668417,77.120303,2.071454,445.4,2016.992908
Mar,6.061640,0.803871,12.434839,709.293662,69.261675,2.447647,698.6,2017.000000
Apr,10.601215,4.645333,16.982667,710.002559,64.274459,2.805385,838.1,2017.000000
May,14.001605,8.596129,20.202581,709.830970,69.726098,2.369585,1601.9,2017.000000
Jun,18.983879,13.187333,25.584000,711.072269,68.044504,2.257706,1666.0,2017.000000
Jul,19.910417,14.322581,26.470323,711.496566,70.569659,1.934507,1780.3,2017.000000
Aug,19.554905,13.970323,26.247097,712.556892,72.964836,2.022288,2318.1,2017.000000
Sep,14.570158,9.645333,20.586000,712.863897,76.828923,1.865618,1394.3,2017.000000


# statistical comparisons

In [23]:


# Analysis of Variance Test (ANOVA) 
from scipy.stats import f_oneway

#compare mean temperatures between innsbruck and london
stat, p = f_oneway(inndf['precip_sum (mm)'], londf['precip_sum (mm)'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Not Significantly different')
else:
	print('Significantly different')


stat=159.050, p=0.000
Significantly different
